## Project 2 - Group 11
### Aditya Mohan Kakde, Venkata Subrahmanya Ankit Vennelakanti, Venkata Krishnan Ravichandran
#### Task 2 - Permutation Entropy and Complexity

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

In [ ]:
''' This module has essential functions supporting
fast and effective computation of permutation entropy and
its different variations.'''
import numpy as np


def s_entropy(freq_list):
    ''' This function computes the shannon entropy of a given frequency distribution.
    USAGE: shannon_entropy(freq_list)
    ARGS: freq_list = Numeric vector representing the frequency distribution
    OUTPUT: A numeric value representing shannon's entropy'''
    freq_list = [element for element in freq_list if element != 0]
    sh_entropy = 0.0
    for freq in freq_list:
        sh_entropy += freq * np.log(freq)
    sh_entropy = -sh_entropy
    return(sh_entropy)

def ordinal_patterns(ts, embdim, embdelay):
    ''' This function computes the ordinal patterns of a time series for a given embedding dimension and embedding delay.
    USAGE: ordinal_patterns(ts, embdim, embdelay)
    ARGS: ts = Numeric vector representing the time series, embdim = embedding dimension (3<=embdim<=7 prefered range), embdelay =  embdding delay
    OUPTUT: A numeric vector representing frequencies of ordinal patterns'''
    m, t = embdim, embdelay
    x = ts if isinstance(ts, np.ndarray) else np.array(ts) 

    tmp = np.zeros((x.shape[0], m))
    for i in range(m):
        tmp[:, i] = np.roll(x, i*t)
    partition = tmp[(t*m-1):, :] 
    permutation = np.argsort(partition)
    idx = _hash(permutation)

    counts = np.zeros(np.math.factorial(m))
    for i in range(counts.shape[0]):
        counts[i] = (idx == i).sum()
    return list(counts[counts != 0].astype(int))

def _hash(x):
    m, n = x.shape
    if n == 1:
        return np.zeros(m)
    return np.sum(np.apply_along_axis(lambda y: y < x[:, 0], 0, x), axis=1) * np.math.factorial(n-1) + _hash(x[:, 1:]) 
    

def p_entropy(op):
    ordinal_pat = op
    max_entropy = np.log(len(ordinal_pat))
    p = np.divide(np.array(ordinal_pat), float(sum(ordinal_pat)))
    return(s_entropy(p)/max_entropy)

def complexity(op):
    ''' This function computes the complexity of a time series defined as: Comp_JS = Q_o * JSdivergence * pe
    Q_o = Normalizing constant
    JSdivergence = Jensen-Shannon divergence
    pe = permutation entopry
    ARGS: ordinal pattern'''
    pe = p_entropy(op)
    constant1 = (0.5+((1 - 0.5)/len(op)))* np.log(0.5+((1 - 0.5)/len(op)))
    constant2 = ((1 - 0.5)/len(op))*np.log((1 - 0.5)/len(op))*(len(op) - 1)
    constant3 = 0.5*np.log(len(op))
    Q_o = -1/(constant1+constant2+constant3)

    temp_op_prob = np.divide(op, sum(op))
    temp_op_prob2 = (0.5*temp_op_prob)+(0.5*(1/len(op)))
    JSdivergence = (s_entropy(temp_op_prob2) - 0.5 * s_entropy(temp_op_prob) - 0.5 * np.log(len(op)))
    Comp_JS = Q_o * JSdivergence * pe
    return(Comp_JS)

def weighted_ordinal_patterns(ts, embdim, embdelay):
    m, t = embdim, embdelay
    x = ts if isinstance(ts, np.ndarray) else np.array(ts) 

    tmp = np.zeros((x.shape[0], m))
    for i in range(m):
        tmp[:, i] = np.roll(x, i*t)
    partition = tmp[(t*m-1):, :] 
    xm = np.mean(partition, axis=1)
    weight = np.mean((partition - xm[:, None])**2, axis=1)
    permutation = np.argsort(partition)
    idx = _hash(permutation)
    counts = np.zeros(np.math.factorial(m))
    for i in range(counts.shape[0]):
        counts[i] = sum(weight[i == idx])

    return list(counts[counts != 0]) 

We are calculating the resultant vector of x,y,z for all the subjects and all the activities of head and chest sensors and appending all the results to a new dataframe 'df_final'

In [ ]:
from plotly.basewidget import BaseFigureWidget
activities = ['_walking_csv','_running_csv','_climbingdown_csv','_climbingup_csv']
body_part = ['head','chest']

df_final=pd.DataFrame(columns=['id','time','attr_x','attr_y','attr_z','Subject','body_position','Activity','Res_vec'])
for subject_number in range(1,16):
  for activity in range(4):
    for position in range(2):
      #for axis in range(3):
      file_path = 'drive/MyDrive/IE5374 Workspace/Colab Notebooks/Project_2/Data/s' + str(subject_number) + activities[activity] + '/acc' + activities[activity][:-3] + body_part[position] + '.csv'
      df = pd.read_csv(file_path)
      df.columns=['id','time','attr_x','attr_y','attr_z']
      df['Subject']=[subject_number]*df.shape[0]
      df['body_position']=[body_part[position]]*df.shape[0]
      df['Activity']=[activities[activity][1:-3]]*df.shape[0]  
      #Climbing down
      res_vec_sq = (df['attr_x']**2)+(df['attr_y']**2)+(df['attr_z']**2)
      res_vec = res_vec_sq**(1/2)
      df['Res_vec']=res_vec
      df_final=pd.concat([df_final,df])

      df_walking = df_final[df_final['Activity']=='walking_']
      df_running = df_final[df_final['Activity']=='running_']
      df_climbingdown = df_final[df_final['Activity']=='climbingdown_']
      df_climbingup = df_final[df_final['Activity']=='climbingup_']
df_final


,id,time,attr_x,attr_y,attr_z,Subject,body_position,Activity,Res_vec
0,1,1435993159428,-2.891597,9.324817,0.042497,1,head,walking_,9.762958
1,2,1435993159429,-2.911948,9.337985,0.023942,1,head,walking_,9.781512
2,3,1435993159445,-2.894590,9.332000,0.041300,1,head,walking_,9.770700
3,4,1435993159464,-2.859275,9.305663,0.029928,1,head,walking_,9.735077
4,5,1435993159473,-2.823961,9.351752,0.007183,1,head,walking_,9.768832
...,...,...,...,...,...,...,...,...,...
29167,29168,1437227855917,1.139640,9.154829,-3.615245,15,chest,climbingup_,9.908565
29168,29169,1437227855939,1.125873,8.999206,-3.518279,15,chest,climbingup_,9.727877
29169,29170,1437227855959,1.190517,8.949526,-3.393781,15,chest,climbingup_,9.645159
29170,29171,1437227855978,1.173757,8.920197,-3.334524,15,chest,climbingup_,9.595138


#Question 1
Caluclation of using Permutation Entropy,Complexity resultant vector

In [ ]:
j=0
pe = []
comp = []
activity = []
for i in range(1,50):
    op_walking=ordinal_patterns(df_walking['Res_vec'][j:(j+4096)],4,1)
    op_running=ordinal_patterns(df_running['Res_vec'][j:(j+4096)],4,1)
    op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+4096)],4,1)
    op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+4096)],4,1)
    pe.append(p_entropy(op_walking))
    pe.append(p_entropy(op_running))
    pe.append(p_entropy(op_climbingup))
    pe.append(p_entropy(op_climbingdown))
    comp.append(complexity(op_walking))
    comp.append(complexity(op_running))
    comp.append(complexity(op_climbingup))
    comp.append(complexity(op_climbingdown))
    activity.append("Walking")
    activity.append("Running")
    activity.append("Climbing Up")
    activity.append("Climbing down")
    j = j+129

result = pd.DataFrame({'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
result

,Permutation_Entropy,Complexity,Activity
0,0.821378,0.177668,Walking
1,0.747618,0.200744,Running
2,0.741987,0.214150,Climbing Up
3,0.818059,0.175969,Climbing down
4,0.818831,0.179513,Walking
...,...,...,...
191,0.792009,0.191704,Climbing down
192,0.825438,0.176746,Walking
193,0.707985,0.222314,Running
194,0.749101,0.211488,Climbing Up


In [ ]:
res_walking_running = result[(result['Activity']=='Walking') | (result['Activity']=='Running')]
res_walking_running

,Permutation_Entropy,Complexity,Activity
0,0.821378,0.177668,Walking
1,0.747618,0.200744,Running
4,0.818831,0.179513,Walking
5,0.737926,0.205365,Running
8,0.817194,0.180465,Walking
...,...,...,...
185,0.709132,0.221357,Running
188,0.823625,0.177785,Walking
189,0.707006,0.222195,Running
192,0.825438,0.176746,Walking


In [ ]:
res_climbing_up_down = result[(result['Activity']=='Climbing down') | (result['Activity']=='Climbing Up')]
res_climbing_up_down

,Permutation_Entropy,Complexity,Activity
2,0.741987,0.214150,Climbing Up
3,0.818059,0.175969,Climbing down
6,0.730717,0.220126,Climbing Up
7,0.817934,0.176105,Climbing down
10,0.716311,0.227632,Climbing Up
...,...,...,...
187,0.789218,0.193607,Climbing down
190,0.747579,0.212129,Climbing Up
191,0.792009,0.191704,Climbing down
194,0.749101,0.211488,Climbing Up


In [ ]:
import plotly.express as px
fig = px.line(df_walking['Res_vec'], x=range(1,len(df_walking['attr_x'][1000:5096])+1), y=df_walking['Res_vec'][1000:5096])
fig.show()


In [ ]:
import plotly.express as px
fig = px.scatter(x=res_walking_running['Permutation_Entropy'],y=res_walking_running['Complexity'], color=res_walking_running['Activity'])
fig.show()

### For Running and walking - Subject 1

#Question 2


permutation entropy and complexity for walking,running, Climbing up and climbing down and varying parameters like Signal length=4096,2048,1024 embedded delay = 1,2,3 and embedded dimension = 3, 4, 5, 6 (only change d)



Signal Length =4096, embedded delay = 1, embedded dimension = 3

In [ ]:
subject = []
acc_axis = []
signal_len = []
dimension = []
pe = []
comp = []
activity = []
delay = []

In [ ]:
df_walking

,id,time,attr_x,attr_y,attr_z,Subject,body_position,Activity,Res_vec
0,1,1435993159428,-2.891597,9.324817,0.042497,1,head,walking_,9.762958
1,2,1435993159429,-2.911948,9.337985,0.023942,1,head,walking_,9.781512
2,3,1435993159445,-2.894590,9.332000,0.041300,1,head,walking_,9.770700
3,4,1435993159464,-2.859275,9.305663,0.029928,1,head,walking_,9.735077
4,5,1435993159473,-2.823961,9.351752,0.007183,1,head,walking_,9.768832
...,...,...,...,...,...,...,...,...,...
33212,33213,1437229289900,0.532710,9.598953,-1.486799,15,chest,walking_,9.728014
33213,33214,1437229289920,0.586579,9.662998,-1.464653,15,chest,walking_,9.790956
33214,33215,1437229289993,0.578798,9.689933,-1.492186,15,chest,walking_,9.821223
33215,33216,1437229289994,0.594959,9.598354,-1.548450,15,chest,walking_,9.740640


In [ ]:
j=0
subject = []
acc_axis = []
signal_len = []
dimension = []
pe = []
comp = []
activity = []
delay = []
for i in range(1,50):
    op_walking=ordinal_patterns(df_walking['Res_vec'][j:(j+4096)],3,1)
    op_running=ordinal_patterns(df_running['Res_vec'][j:(j+4096)],3,1)
    op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+4096)],3,1)
    op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+4096)],3,1)
    pe.append(p_entropy(op_walking))
    pe.append(p_entropy(op_running))
    pe.append(p_entropy(op_climbingup))
    pe.append(p_entropy(op_climbingdown))
    comp.append(complexity(op_walking))
    comp.append(complexity(op_running))
    comp.append(complexity(op_climbingup))
    comp.append(complexity(op_climbingdown))

    subject.append(df_walking.iloc[i,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('4096')
    dimension.append('3')
    delay.append('1')
    activity.append("Walking")

    subject.append(df_running.iloc[i,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('4096')
    dimension.append('3')
    delay.append('1')
    activity.append("Running")

    subject.append(df_climbingup.iloc[i,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('4096')
    dimension.append('3')
    delay.append('1')
    activity.append("Climbing Up")

    subject.append(df_climbingdown.iloc[i,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('4096')
    dimension.append('3')
    delay.append('1')
    activity.append("Climbing down")
    j = j+129
    
result = pd.DataFrame({'Acclerometer Axis':acc_axis,'Signal Length':signal_len,'Dimension':dimension,'Delay':delay,'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
result

,Acclerometer Axis,Signal Length,Dimension,Delay,Permutation_Entropy,Complexity,Activity
0,Resultant Vector,4096,3,1,0.877399,0.103483,Walking
1,Resultant Vector,4096,3,1,0.808752,0.139451,Running
2,Resultant Vector,4096,3,1,0.812660,0.147121,Climbing Up
3,Resultant Vector,4096,3,1,0.873077,0.106517,Climbing down
4,Resultant Vector,4096,3,1,0.875311,0.104879,Walking
...,...,...,...,...,...,...,...
191,Resultant Vector,4096,3,1,0.853511,0.120455,Climbing down
192,Resultant Vector,4096,3,1,0.879368,0.102462,Walking
193,Resultant Vector,4096,3,1,0.784718,0.156615,Running
194,Resultant Vector,4096,3,1,0.822668,0.140782,Climbing Up


In [ ]:
fig = px.scatter(x=result['Permutation_Entropy'],y=result['Complexity'], color=result['Activity'])
fig.show()

Signal Length =1024, embedded delay = 2, embedded dimension = 4

In [ ]:
j=0
subject = []
acc_axis = []
signal_len = []
dimension = []
pe = []
comp = []
activity = []
delay = []
for i in range(1,50):
    op_walking=ordinal_patterns(df_walking['Res_vec'][j:(j+1024)],4,2)
    op_running=ordinal_patterns(df_running['Res_vec'][j:(j+1024)],4,2)
    op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+1024)],4,2)
    op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+1024)],4,2)
    pe.append(p_entropy(op_walking))
    pe.append(p_entropy(op_running))
    pe.append(p_entropy(op_climbingup))
    pe.append(p_entropy(op_climbingdown))
    comp.append(complexity(op_walking))
    comp.append(complexity(op_running))
    comp.append(complexity(op_climbingup))
    comp.append(complexity(op_climbingdown))

    subject.append(df_walking.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('1024')
    dimension.append('4')
    delay.append('2')
    activity.append("Walking")

    subject.append(df_running.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('1024')
    dimension.append('4')
    delay.append('2')
    activity.append("Running")

    subject.append(df_climbingup.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('1024')
    dimension.append('4')
    delay.append('2')
    activity.append("Climbing Up")


    subject.append(df_climbingdown.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('1024')
    dimension.append('4')
    delay.append('2')
    activity.append("Climbing down")
    j = j+129
    
result = pd.DataFrame({'Acclerometer Axis':acc_axis,'Signal Length':signal_len,'Dimension':dimension,'Delay':delay,'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
result

,Acclerometer Axis,Signal Length,Dimension,Delay,Permutation_Entropy,Complexity,Activity
0,Resultant Vector,1024,4,2,0.905204,0.101714,Walking
1,Resultant Vector,1024,4,2,0.842318,0.144766,Running
2,Resultant Vector,1024,4,2,0.916155,0.090054,Climbing Up
3,Resultant Vector,1024,4,2,0.907408,0.107443,Climbing down
4,Resultant Vector,1024,4,2,0.896160,0.110467,Walking
...,...,...,...,...,...,...,...
191,Resultant Vector,1024,4,2,0.872506,0.124015,Climbing down
192,Resultant Vector,1024,4,2,0.926103,0.088211,Walking
193,Resultant Vector,1024,4,2,0.795054,0.180240,Running
194,Resultant Vector,1024,4,2,0.777303,0.197940,Climbing Up


In [ ]:
fig = px.scatter(x=result['Permutation_Entropy'],y=result['Complexity'], color=result['Activity'])
fig.show()

Signal Length =2048, embedded delay = 3, embedded dimension = 6

In [ ]:
j=0
subject = []
acc_axis = []
signal_len = []
dimension = []
pe = []
comp = []
activity = []
delay = []
for i in range(1,50):
    op_walking=ordinal_patterns(df_walking['Res_vec'][j:(j+2048)],6,3)
    op_running=ordinal_patterns(df_running['Res_vec'][j:(j+2048)],6,3)
    op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+2048)],6,3)
    op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+2048)],6,3)
    pe.append(p_entropy(op_walking))
    pe.append(p_entropy(op_running))
    pe.append(p_entropy(op_climbingup))
    pe.append(p_entropy(op_climbingdown))
    comp.append(complexity(op_walking))
    comp.append(complexity(op_running))
    comp.append(complexity(op_climbingup))
    comp.append(complexity(op_climbingdown))

    subject.append(df_walking.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('2048')
    dimension.append('6')
    delay.append('3')
    activity.append("Walking")

    subject.append(df_running.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('2048')
    dimension.append('6')
    delay.append('3')
    activity.append("Running")

    subject.append(df_climbingup.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('2048')
    dimension.append('6')
    delay.append('3')
    activity.append("Climbing Up")


    subject.append(df_climbingdown.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('2048')
    dimension.append('6')
    delay.append('3')
    activity.append("Climbing down")
    j = j+129
    
result = pd.DataFrame({'Acclerometer Axis':acc_axis,'Signal Length':signal_len,'Dimension':dimension,'Delay':delay,'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
result

,Acclerometer Axis,Signal Length,Dimension,Delay,Permutation_Entropy,Complexity,Activity
0,Resultant Vector,2048,6,3,0.913723,0.159456,Walking
1,Resultant Vector,2048,6,3,0.830555,0.262482,Running
2,Resultant Vector,2048,6,3,0.859881,0.205706,Climbing Up
3,Resultant Vector,2048,6,3,0.915848,0.150843,Climbing down
4,Resultant Vector,2048,6,3,0.914041,0.159355,Walking
...,...,...,...,...,...,...,...
191,Resultant Vector,2048,6,3,0.867312,0.215833,Climbing down
192,Resultant Vector,2048,6,3,0.924632,0.145962,Walking
193,Resultant Vector,2048,6,3,0.857859,0.208575,Running
194,Resultant Vector,2048,6,3,0.838954,0.231568,Climbing Up


In [ ]:
fig = px.scatter(x=result['Permutation_Entropy'],y=result['Complexity'], color=result['Activity'])
fig.show()

Signal Length =2048, embedded delay = 2, embedded dimension = 5

In [ ]:
j=0
subject = []
acc_axis = []
signal_len = []
dimension = []
pe = []
comp = []
activity = []
delay = []
for i in range(1,50):
    op_walking=ordinal_patterns(df_walking['Res_vec'][j:(j+2048)],5,2)
    op_running=ordinal_patterns(df_running['Res_vec'][j:(j+2048)],5,2)
    op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+2048)],5,2)
    op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+2048)],5,2)
    pe.append(p_entropy(op_walking))
    pe.append(p_entropy(op_running))
    pe.append(p_entropy(op_climbingup))
    pe.append(p_entropy(op_climbingdown))
    comp.append(complexity(op_walking))
    comp.append(complexity(op_running))
    comp.append(complexity(op_climbingup))
    comp.append(complexity(op_climbingdown))

    subject.append(df_walking.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('2048')
    dimension.append('5')
    delay.append('2')
    activity.append("Walking")

    subject.append(df_running.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('2048')
    dimension.append('5')
    delay.append('2')
    activity.append("Running")

    subject.append(df_climbingup.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('2048')
    dimension.append('5')
    delay.append('2')
    activity.append("Climbing Up")


    subject.append(df_climbingdown.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('2048')
    dimension.append('5')
    delay.append('2')
    activity.append("Climbing down")
    j = j+129
    
result = pd.DataFrame({'Acclerometer Axis':acc_axis,'Signal Length':signal_len,'Dimension':dimension,'Delay':delay,'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
result

,Acclerometer Axis,Signal Length,Dimension,Delay,Permutation_Entropy,Complexity,Activity
0,Resultant Vector,2048,5,2,0.885375,0.161824,Walking
1,Resultant Vector,2048,5,2,0.789125,0.229039,Running
2,Resultant Vector,2048,5,2,0.818665,0.196690,Climbing Up
3,Resultant Vector,2048,5,2,0.869158,0.181511,Climbing down
4,Resultant Vector,2048,5,2,0.879982,0.170010,Walking
...,...,...,...,...,...,...,...
191,Resultant Vector,2048,5,2,0.817479,0.217260,Climbing down
192,Resultant Vector,2048,5,2,0.905157,0.143295,Walking
193,Resultant Vector,2048,5,2,0.785453,0.226887,Running
194,Resultant Vector,2048,5,2,0.759557,0.256306,Climbing Up


In [ ]:
fig = px.scatter(x=result['Permutation_Entropy'],y=result['Complexity'], color=result['Activity'])
fig.show()

Signal Length =2048, embedded delay = 1, embedded dimension = 6

In [ ]:
j=0
subject = []
acc_axis = []
signal_len = []
dimension = []
pe = []
comp = []
activity = []
delay = []
for i in range(1,50):
    op_walking=ordinal_patterns(df_walking['Res_vec'][j:(j+2048)],6,1)
    op_running=ordinal_patterns(df_running['Res_vec'][j:(j+2048)],6,1)
    op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+2048)],6,1)
    op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+2048)],6,1)
    pe.append(p_entropy(op_walking))
    pe.append(p_entropy(op_running))
    pe.append(p_entropy(op_climbingup))
    pe.append(p_entropy(op_climbingdown))
    comp.append(complexity(op_walking))
    comp.append(complexity(op_running))
    comp.append(complexity(op_climbingup))
    comp.append(complexity(op_climbingdown))

    subject.append(df_walking.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('2048')
    dimension.append('6')
    delay.append('1')
    activity.append("Walking")

    subject.append(df_running.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('2048')
    dimension.append('6')
    delay.append('1')
    activity.append("Running")

    subject.append(df_climbingup.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('2048')
    dimension.append('6')
    delay.append('1')
    activity.append("Climbing Up")


    subject.append(df_climbingdown.iloc[j,5])
    acc_axis.append("Resultant Vector")
    signal_len.append('2048')
    dimension.append('6')
    delay.append('1')
    activity.append("Climbing down")
    j = j+129
    
result = pd.DataFrame({'Acclerometer Axis':acc_axis,'Signal Length':signal_len,'Dimension':dimension,'Delay':delay,'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
result

,Acclerometer Axis,Signal Length,Dimension,Delay,Permutation_Entropy,Complexity,Activity
0,Resultant Vector,2048,6,1,0.837940,0.224180,Walking
1,Resultant Vector,2048,6,1,0.742496,0.233659,Running
2,Resultant Vector,2048,6,1,0.755146,0.245160,Climbing Up
3,Resultant Vector,2048,6,1,0.827019,0.223147,Climbing down
4,Resultant Vector,2048,6,1,0.835360,0.224332,Walking
...,...,...,...,...,...,...,...
191,Resultant Vector,2048,6,1,0.802537,0.227844,Climbing down
192,Resultant Vector,2048,6,1,0.849887,0.212344,Walking
193,Resultant Vector,2048,6,1,0.724666,0.242800,Running
194,Resultant Vector,2048,6,1,0.744140,0.265926,Climbing Up


In [ ]:
fig = px.scatter(x=result['Permutation_Entropy'],y=result['Complexity'], color=result['Activity'])
fig.show()

#Question 3

Permutation entropy vs complexity and color the points according to Walking and Running where Signal length = 4096, embedded delay = 1 and embedded dimension = 3,4,5,6 (only change d)

Signal Length =4096, embedded delay = 1, embedded dimension = 3

In [ ]:
import plotly.express as px
j=0
pe = []
comp = []
activity = []
for i in range(1,50):
    op_walking=ordinal_patterns(df_walking['Res_vec'][j:(j+4096)],3,1)
    op_running=ordinal_patterns(df_running['Res_vec'][j:(j+4096)],3,1)
    #op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+4096)],3,1)
    #op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+4096)],3,1)
    pe.append(p_entropy(op_walking))
    pe.append(p_entropy(op_running))
    #pe.append(p_entropy(op_climbingup))
    #pe.append(p_entropy(op_climbingdown))
    comp.append(complexity(op_walking))
    comp.append(complexity(op_running))
    #comp.append(complexity(op_climbingup))
    #comp.append(complexity(op_climbingdown))
    activity.append("Walking")
    activity.append("Running")
    #activity.append("Climbing Up")
    #activity.append("Climbing down")
    j = j+129

result0 = pd.DataFrame({'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
fig = px.scatter(x=result0['Permutation_Entropy'],y=result0['Complexity'], color=result0['Activity'])
fig.show()

Signal Length =4096, embedded delay = 1, embedded dimension = 4

In [ ]:
import plotly.express as px
j=0
pe = []
comp = []
activity = []
for i in range(1,50):
    op_walking=ordinal_patterns(df_walking['Res_vec'][j:(j+4096)],4,1)
    op_running=ordinal_patterns(df_running['Res_vec'][j:(j+4096)],4,1)
    #op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+4096)],3,1)
    #op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+4096)],3,1)
    pe.append(p_entropy(op_walking))
    pe.append(p_entropy(op_running))
    #pe.append(p_entropy(op_climbingup))
    #pe.append(p_entropy(op_climbingdown))
    comp.append(complexity(op_walking))
    comp.append(complexity(op_running))
    #comp.append(complexity(op_climbingup))
    #comp.append(complexity(op_climbingdown))
    activity.append("Walking")
    activity.append("Running")
    #activity.append("Climbing Up")
    #activity.append("Climbing down")
    j = j+129

result0 = pd.DataFrame({'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
fig = px.scatter(x=result0['Permutation_Entropy'],y=result0['Complexity'], color=result0['Activity'])
fig.show()

Signal Length =4096, embedded delay = 1, embedded dimension = 5

In [ ]:
import plotly.express as px
j=0
pe = []
comp = []
activity = []
for i in range(1,50):
    op_walking=ordinal_patterns(df_walking['Res_vec'][j:(j+4096)],5,1)
    op_running=ordinal_patterns(df_running['Res_vec'][j:(j+4096)],5,1)
    #op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+4096)],3,1)
    #op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+4096)],3,1)
    pe.append(p_entropy(op_walking))
    pe.append(p_entropy(op_running))
    #pe.append(p_entropy(op_climbingup))
    #pe.append(p_entropy(op_climbingdown))
    comp.append(complexity(op_walking))
    comp.append(complexity(op_running))
    #comp.append(complexity(op_climbingup))
    #comp.append(complexity(op_climbingdown))
    activity.append("Walking")
    activity.append("Running")
    #activity.append("Climbing Up")
    #activity.append("Climbing down")
    j = j+129

result0 = pd.DataFrame({'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
fig = px.scatter(x=result0['Permutation_Entropy'],y=result0['Complexity'], color=result0['Activity'])
fig.show()

Signal Length =4096, embedded delay = 1, embedded dimension = 6

In [ ]:
import plotly.express as px
j=0
pe = []
comp = []
activity = []
for i in range(1,50):
    op_walking=ordinal_patterns(df_walking['Res_vec'][j:(j+4096)],6,1)
    op_running=ordinal_patterns(df_running['Res_vec'][j:(j+4096)],6,1)
    #op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+4096)],3,1)
    #op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+4096)],3,1)
    pe.append(p_entropy(op_walking))
    pe.append(p_entropy(op_running))
    #pe.append(p_entropy(op_climbingup))
    #pe.append(p_entropy(op_climbingdown))
    comp.append(complexity(op_walking))
    comp.append(complexity(op_running))
    #comp.append(complexity(op_climbingup))
    #comp.append(complexity(op_climbingdown))
    activity.append("Walking")
    activity.append("Running")
    #activity.append("Climbing Up")
    #activity.append("Climbing down")
    j = j+129

result0 = pd.DataFrame({'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
fig = px.scatter(x=result0['Permutation_Entropy'],y=result0['Complexity'], color=result0['Activity'])
fig.show()

#Question 4

Permutation entropy vs complexity and color the points according to climbing up and climbing down Signal length = 4096, embedded delay = 1 and embedded dimension = 3, 4, 5, 6 (only change d)

Signal Length =4096, embedded delay = 1, embedded dimension = 3

In [ ]:
import plotly.express as px
j=0
pe = []
comp = []
activity = []
for i in range(1,50):
    
    op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+4096)],3,1)
    op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+4096)],3,1)
 
    pe.append(p_entropy(op_climbingup))
    pe.append(p_entropy(op_climbingdown))
  
    comp.append(complexity(op_climbingup))
    comp.append(complexity(op_climbingdown))

    activity.append("Climbing Up")
    activity.append("Climbing down")
    j = j+129

result0 = pd.DataFrame({'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
fig = px.scatter(x=result0['Permutation_Entropy'],y=result0['Complexity'], color=result0['Activity'])
fig.show()

Signal Length =4096, embedded delay = 1, embedded dimension = 4

In [ ]:
import plotly.express as px
j=0
pe = []
comp = []
activity = []
for i in range(1,50):

    op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+4096)],4,1)
    op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+4096)],4,1)

    pe.append(p_entropy(op_climbingup))
    pe.append(p_entropy(op_climbingdown))

    comp.append(complexity(op_climbingup))
    comp.append(complexity(op_climbingdown))

    activity.append("Climbing Up")
    activity.append("Climbing down")
    j = j+129

result1 = pd.DataFrame({'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
fig = px.scatter(x=result1['Permutation_Entropy'],y=result1['Complexity'], color=result1['Activity'])
fig.show()

Signal Length =4096, embedded delay = 1, embedded dimension = 5

In [ ]:
j=0
pe = []
comp = []
activity = []
for i in range(1,50):

    op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+4096)],5,1)
    op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+4096)],5,1)

    pe.append(p_entropy(op_climbingup))
    pe.append(p_entropy(op_climbingdown))

    comp.append(complexity(op_climbingup))
    comp.append(complexity(op_climbingdown))

    activity.append("Climbing Up")
    activity.append("Climbing down")
    j = j+129

resulty = pd.DataFrame({'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
import plotly.express as px
fig = px.scatter(x=resulty['Permutation_Entropy'],y=resulty['Complexity'], color=resulty['Activity'])
fig.show()

Signal Length =4096, embedded delay = 1, embedded dimension = 6

In [ ]:
j=0
pe = []
comp = []
activity = []
for i in range(1,50):

    op_climbingup=ordinal_patterns(df_climbingup['Res_vec'][j:(j+4096)],6,1)
    op_climbingdown=ordinal_patterns(df_climbingdown['Res_vec'][j:(j+4096)],6,1)

    pe.append(p_entropy(op_climbingup))
    pe.append(p_entropy(op_climbingdown))

    comp.append(complexity(op_climbingup))
    comp.append(complexity(op_climbingdown))

    activity.append("Climbing Up")
    activity.append("Climbing down")
    j = j+129

resultx = pd.DataFrame({'Permutation_Entropy':pe,'Complexity':comp,'Activity':activity})
import plotly.express as px
fig = px.scatter(x=resultx['Permutation_Entropy'],y=resultx['Complexity'], color=resultx['Activity'])
fig.show()